In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [12]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [8]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [9]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [11]:
# initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.3303,  0.3255, -0.4488,  ...,  0.1155,  0.4615,  0.9316],
        [-0.1027, -0.6424, -0.4032,  ...,  0.4605,  2.1825,  0.3425],
        [-0.1665,  1.5384, -0.3312,  ...,  0.7681, -1.0083,  0.6301],
        ...,
        [ 0.1069,  0.8268,  1.2518,  ..., -1.3152, -2.8423,  0.6586],
        [-0.6342,  1.2379, -1.4351,  ..., -0.0283,  0.0826, -1.2305],
        [ 0.6733, -0.7177, -0.5025,  ..., -1.5074,  0.5423, -1.2292]],
       requires_grad=True)

In [13]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)  # softmax is internally computed
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [14]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost = cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.003525567
Epoch: 0002 cost = 0.001746130
Epoch: 0003 cost = 0.001107556
Epoch: 0004 cost = 0.000847621
Epoch: 0005 cost = 0.001633248
Epoch: 0006 cost = 0.001473200
Epoch: 0007 cost = 0.000781952
Epoch: 0008 cost = 0.001062899
Epoch: 0009 cost = 0.000606681
Epoch: 0010 cost = 0.000361466
Epoch: 0011 cost = 0.000769268
Epoch: 0012 cost = 0.001141709
Epoch: 0013 cost = 0.001093609
Epoch: 0014 cost = 0.001157390
Epoch: 0015 cost = 0.000608843
Learning finished


In [24]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8873000144958496
Label:  8
Prediction:  8
